# Generating Pareto Fronts with Uncertainty Predictions for Multi-objective Optimization Problems Using Gaussian Processes
This notebook demonstrates an approach to generate pareto fronts for multi-objective optimization problems with uncertainty predictions using Gaussian process pregression as a surrogate model. These uncertainty predictions are then used to choose additional points to evaluate the objective function in order to improve the Pareto front prediction. The algorithm is outlined below:

1.	The objective function is sampled systematically by an experimental design
2.	A Gaussian process regression analysis is used to model the objective function and the Gaussian process hyperparameters are chosen by maximizing the log marginal likelihood
3.	The Pareto front is generated by minimizing or maximizing one of the objective functions while holding the others fixed over a range of values determined by the desired number of Pareto points
4.	The Pareto front is plotted with error ellipses showing the uncertainty for each of the Pareto front points
5.	The original objective function is resampled at one or two points with the highest uncertainty and steps 1 through 4 are repeated until the Pareto front is obtained to a sufficient accuracy

The proposed technique is applied to the optimization of a ABS plastic panel (shown below) where the input parameters are the material thickness and the height of the stiffening ribs. The output parameters are the mass of the panel and the deflection of the panel under a central load. The goal is to minimize both the mass and the deflection.

![Panel](./panel.png) ![Input Parameters](./panel_inputs.png)

In [1]:
from math import *

import numpy as np

from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel
from sklearn.preprocessing import MaxAbsScaler, StandardScaler, MinMaxScaler
from sklearn.pipeline import Pipeline

import pandas as pd

from scipy.optimize import minimize

from os.path import splitext,basename,join

from bokeh.plotting import figure
from bokeh.models import ColumnDataSource, HoverTool
from bokeh.io import output_file, output_notebook, show, vplot
from bokeh.models.layouts import Column

In [2]:
output_notebook()

Loading BokehJS ...

In [3]:
# Patch predict method of Pipeline to handle keyword arguements
# and to allow transformations on y in addition to x

def predict(self, X, **kwargs):
    """Applies transforms to the data, and the predict method of the
    final estimator. Valid only if the final estimator implements
    predict."""
    Xt = X
    for name, transform in self.steps[:-1]:
        Xt = transform.transform(Xt)

    if self.y_transform is None:
        return self.steps[-1][-1].predict(Xt, **kwargs)
    elif kwargs.get('return_std'):
        return (self.y_transform.inverse_transform((self.steps[-1][-1].predict(Xt, **kwargs)[0]).reshape(-1,1))[:,0],
                self.steps[-1][-1].predict(Xt, **kwargs)[1]/abs(self.y_transform.scale_[0]))
    else:
        return self.y_transform.inverse_transform(self.steps[-1][-1].predict(Xt, **kwargs).reshape(-1,1))[:,0]

Pipeline.predict = predict

In [4]:
# Load datafile that contains the design of experiment runs of the original objective function

#data_file = 'panel_data.xlsx'
#data_file = 'bottle_data.xlsx'
#data_file = 'panel_data_with_2_extra_runs.xlsx'
data_file = 'panel_data.xlsx'
num_pareto_points = 100
#method='SLSQP'
method='COBYLA'

In [5]:
df = pd.read_excel(data_file,header=(0,1,2,3),index_col=0)

data = df.get_values()

input_data = df.xs('input',axis=1,level='type')
input_labels = input_data.columns.get_level_values(0).values

output_data = df.xs('output',axis=1,level='type')
output_labels = output_data.columns.get_level_values(0).values

x_ranges = input_data.max().get_values()-input_data.min().get_values()

df

name,material thickness (mm),rib height (mm),displacement (mm),mass (g)
type,input,input,output,output
goal,Unnamed: 1_level_2,Unnamed: 2_level_2,min,min
pareto axis,Unnamed: 1_level_3,Unnamed: 2_level_3,x,y
1,7.5,15,0.87797,1270.980
2,2.5,15,7.86636,426.956
3,7.5,7,1.46584,1101.300
4,2.5,7,22.21679,368.753
5,7.5,11,1.13536,1186.140
6,2.5,11,12.27396,397.855
7,5.0,15,2.14511,850.594


In [6]:
doe_df = df.copy()
doe_df.columns = doe_df.columns.get_level_values(0)

In [7]:
doe_df

name,material thickness (mm),rib height (mm),displacement (mm),mass (g)
1,7.5,15,0.87797,1270.980
2,2.5,15,7.86636,426.956
3,7.5,7,1.46584,1101.300
4,2.5,7,22.21679,368.753
5,7.5,11,1.13536,1186.140
6,2.5,11,12.27396,397.855
7,5.0,15,2.14511,850.594
8,5.0,7,4.30295,735.856
9,5.0,11,2.93995,793.225


In [8]:
# define input data features
X = input_data.get_values()

# define the target values
y = output_data.get_values()

In [9]:
# create a separate model for each output
models = []

for output in y.T:
    
    #kernel = RBF(length_scale=1.0)
    #kernel = ConstantKernel(1.0, constant_value_bounds=(0.1,10))*RBF(length_scale=(1.0,1.0), length_scale_bounds=(.01,1000))
    kernel = RBF(length_scale=(2.0,2.0), length_scale_bounds=(.01,1000))
    gp_regressor = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=100)

    #standard_scaler = StandardScaler(copy=True, with_mean=True, with_std=True)
    minmax_scaler = MinMaxScaler(feature_range=(-1,1))

    model = Pipeline([("transform", minmax_scaler),
                      ("gp_regressor", gp_regressor)])
    
    #model.y_transform = StandardScaler(copy=True, with_mean=True, with_std=True)
    #model.y_transform = MaxAbsScaler()
    model.y_transform = MinMaxScaler(feature_range=(-1,1))
    output = model.y_transform.fit_transform(output.reshape(-1,1))[:,0]

    model.fit(X, output)
    
    models.append(model)
    
    print(np.exp(model.named_steps["gp_regressor"].kernel_.theta))


[ 0.75186248  1.96709693]
[  2.06781866  14.19036394]


C:\Users\micha\Anaconda3\lib\site-packages\sklearn\gaussian_process\gpr.py:427: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'task': b'ABNORMAL_TERMINATION_IN_LNSRCH', 'funcalls': 53, 'warnflag': 2, 'grad': array([  4.72226238e-05,  -9.83554037e-06]), 'nit': 11}
  " state: %s" % convergence_dict)


In [10]:
# determine pareto axes and axes goals
y_axis_df = output_data.xs('y',axis=1,level='pareto axis')
x_axis_df = output_data.xs('x',axis=1,level='pareto axis')

y_axis_label = y_axis_df.columns.get_level_values(0).values[0]
y_axis_index = output_data.columns.get_level_values(0).get_loc(y_axis_label)
y_axis_goal = y_axis_df.columns.get_level_values(1)

x_axis_label = x_axis_df.columns.get_level_values(0).values[0]
x_axis_index = output_data.columns.get_level_values(0).get_loc(x_axis_label)
x_axis_goal = x_axis_df.columns.get_level_values(1)


In [11]:
# setup the inequality constraints for each input (use input data ranges)
# also, add any equality constraints that have been specified for the input values
constraints = []

for index,column in enumerate(input_data):
    if type(column[1]) is float or type(column[1]) is int:
        # equality constraint
        constraints.append({'type': 'eq',
                            'fun' : lambda x,target=column[1]: x[index]-target})
    
    else:
        # inequality constraint
    
        lower_lim = min(input_data[column])
        upper_lim = max(input_data[column])

        constraints.append({'type': 'ineq',
                            'fun' : lambda x,lower_lim=lower_lim,index=index: x[index]-lower_lim})
        constraints.append({'type': 'ineq',
                            'fun' : lambda x,upper_lim=upper_lim,index=index: upper_lim-x[index]})
    

In [12]:
# add equality constraints for any that are specified for the outputs
for index,column in enumerate(output_data):
    if type(column[1]) is float or type(column[1]) is int:
        print(index)
        # equality constraint
        constraints.append({'type': 'eq',
                            'fun' : lambda x,target=column[1],index=index: models[index].predict(x.reshape(1,-1))-target})


In [13]:
def objective_func(x,index,sign=1.0): 
    return sign*models[index].predict(x.reshape(1,-1))

# first, get the x pareto value lower and upper limits

num_func_evaluations = 0

current_min = None
for x_starting_point in X:
    res = minimize(objective_func, x_starting_point, args=(x_axis_index,-1.0), 
                   constraints=constraints, method=method, options={'disp': False})
    
    num_func_evaluations += res.nfev
    
    if current_min is None:
        current_min = res.fun
    elif res.fun < current_min:
        current_min = res.fun
    
x_max = -current_min
    
current_min = None
for x_starting_point in X:

    res = minimize(objective_func, x_starting_point, args=(x_axis_index,1.0),
                   constraints=constraints, method=method, options={'disp': False})
    
    num_func_evaluations += res.nfev
    
    if current_min is None:
        current_min = res.fun
    elif res.fun < current_min:
        current_min = res.fun
        
x_min = current_min

pareto_points = np.linspace(x_min,x_max,num=num_pareto_points)



In [14]:
# For each x-value in pareto_points, the remaining objective is minimized or maximzed 
# to generate the Pareto front

pareto_input_values = []

x_axis_sign = 1 if x_axis_goal == 'min' else -1
y_axis_sign = 1 if y_axis_goal == 'min' else -1

for x_value in pareto_points:
    current_constraint = {'type': 'ineq',
           'fun' : lambda x, sign=x_axis_sign, limit=x_value, index=x_axis_index : 
                          sign*(limit-models[index].predict(x.reshape(1,-1)))}
    
    min_objective_value = None
    
    for x_starting_point in X:
    
        res = minimize(objective_func, x_starting_point, args=(y_axis_index,y_axis_sign),
                       constraints=constraints+[current_constraint,], method=method, options={'disp': False})
        
        num_func_evaluations += res.nfev

        if min_objective_value is None:
            best_x = res.x
            best_x_objective = res.fun
        elif res.fun < best_x_objective:
            best_x = res.x
            best_x_objective = res.fun

    pareto_input_values.append(best_x)
        
    
    

In [15]:
# Evaluate each of the objective function models at the Pareto points

pareto_output_values = None
pareto_output_std = None

for model in models:
    y_mean, y_std = model.predict(pareto_input_values, return_std=True)
    
    if pareto_output_values is None:
        pareto_output_values = y_mean
    else:
        pareto_output_values = np.column_stack( (pareto_output_values, y_mean) )
    
    if pareto_output_std is None:
        pareto_output_std = y_std
    else:
        pareto_output_std = np.column_stack( (pareto_output_std, y_std) )
    

C:\Users\micha\Anaconda3\lib\site-packages\sklearn\gaussian_process\gpr.py:308: UserWarning: Predicted variances smaller than 0. Setting those variances to 0.
  warnings.warn("Predicted variances smaller than 0. "


In [16]:
# Create an interactive plot of the Perato points
# Hover mouse to see input and out values at each point

TOOLS = "pan,wheel_zoom,box_zoom,reset,save"
TITLE = "Pareto Data Points"

std_prefix = "4σ "

p = figure(tools=TOOLS, toolbar_location="above", plot_width=800,
           plot_height=400, title=TITLE)

p.xaxis.axis_label = x_axis_label
p.yaxis.axis_label = y_axis_label

original_datapoints = ColumnDataSource(doe_df)

pareto_df = pd.DataFrame(np.hstack( (pareto_input_values, pareto_output_values, 4*pareto_output_std) ), 
                         columns=np.hstack( (input_labels,output_labels,
                                             [std_prefix+label for label in output_labels]) ))

pareto_datapoints = ColumnDataSource(pareto_df)

p.ellipse(x_axis_label, y_axis_label, std_prefix+x_axis_label, 
          std_prefix+y_axis_label, source=pareto_datapoints,
          legend="Pareto +/- 2σ Error Bounds", 
          fill_color='lightgrey', line_color='lightgrey')

p.circle(x_axis_label, y_axis_label, size=6, source=pareto_datapoints,
         line_color="black", fill_alpha=1.0, legend="Pareto Points")

# verification_df = pd.read_excel(splitext(data_file)[0]+'_sw_verification.xlsx')
# verification_datapoints = ColumnDataSource(verification_df)
# p.triangle(x_axis_label, y_axis_label, size=10, source=verification_datapoints,
#          fill_color='magenta', line_color="black", fill_alpha=1.0,
#          legend="FE Verification Runs")

p.diamond(x_axis_label, y_axis_label, size=12, source=original_datapoints,
         fill_color='yellow', line_color="black", fill_alpha=1.0,
         legend="Original FE Runs")

tooltips = [(value,'@{'+value+'}') for value in pareto_df.columns]

hover = HoverTool(
        tooltips=tooltips
    )

p.add_tools(hover)

show(p)

In [17]:
pareto_df.to_excel(splitext(data_file)[0]+'_pareto.xlsx')

In [18]:
print("Number of response surface evaluations to generate Pareto points =",num_func_evaluations)

time_per_fea_run = (8*60)/102

print("Would require",(time_per_fea_run*num_func_evaluations)/60**2,"hours for an FEA run time of",
      time_per_fea_run,"sec per iteration.")

Number of response surface evaluations to generate Pareto points = 27856
Would require 36.41307189542484 hours for an FEA run time of 4.705882352941177 sec per iteration.


## Progressive runs of Pareto front generation algorithm with finite element verification runs
Note that as a relatively small number of points are added to the Gaussian process fit, the uncertainty of the Pareto front points are dramatically reduced. Additionally, the finite element verification runs of the Pareto front points converge to the Pareto front points predicted by the Gaussian process model.

In [19]:
data_files = ['panel_data.xlsx', 'panel_data_with_2_extra_runs.xlsx', 'panel_data_with_3_extra_runs.xlsx',
              'panel_data_with_4_extra_runs.xlsx']

plot_list = []

for data_file in data_files:
    
    df = pd.read_excel(data_file,header=(0,1,2,3),index_col=0)

    doe_df = df.copy()
    doe_df.columns = doe_df.columns.get_level_values(0)
    
    TOOLS = "pan,wheel_zoom,box_zoom,reset,save"
    TITLE = "Pareto Data Points for "+data_file

    std_prefix = "4σ "

    p = figure(tools=TOOLS, toolbar_location="above", plot_width=800,
               plot_height=400, title=TITLE)
    
    p.xaxis.axis_label = x_axis_label
    p.yaxis.axis_label = y_axis_label

    original_datapoints = ColumnDataSource(doe_df)

    pareto_df = pd.read_excel(splitext(data_file)[0]+'_pareto.xlsx')

    pareto_datapoints = ColumnDataSource(pareto_df)

    p.ellipse(x_axis_label, y_axis_label, std_prefix+x_axis_label, 
              std_prefix+y_axis_label, source=pareto_datapoints,
              legend="Pareto +/- 2σ Error Bounds", 
              fill_color='lightgrey', line_color='lightgrey')

    p.circle(x_axis_label, y_axis_label, size=6, source=pareto_datapoints,
             line_color="black", fill_alpha=1.0, legend="Pareto Points")

    verification_df = pd.read_excel(splitext(data_file)[0]+'_sw_verification.xlsx')
    verification_datapoints = ColumnDataSource(verification_df)
    p.triangle(x_axis_label, y_axis_label, size=10, source=verification_datapoints,
             fill_color='magenta', line_color="black", fill_alpha=1.0,
             legend="FE Verification Runs")

    p.diamond(x_axis_label, y_axis_label, size=12, source=original_datapoints,
             fill_color='yellow', line_color="black", fill_alpha=1.0,
             legend="Original FE Runs")

    tooltips = [(value,'@{'+value+'}') for value in pareto_df.columns]

    hover = HoverTool(
            tooltips=tooltips
        )

    p.add_tools(hover)

    plot_list.append(p)

p = Column(*plot_list)
show(p)